<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%209%20-%20Validación%20de%20datos%20con%20TensorFlow%20Data%20Validation%20(TFDV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1:Instalar todas las dependencias y configurar el entorno

In [1]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 0s (35.6 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 144618 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [2]:
!pip install -q --upgrade tensorflow_data_validation

     |████████████████████████████████| 1.3MB 3.3MB/s 
     |████████████████████████████████| 296kB 13.3MB/s 
     |████████████████████████████████| 378kB 19.0MB/s 
     |████████████████████████████████| 8.6MB 22.0MB/s 
     |████████████████████████████████| 1.8MB 62.7MB/s 
     |████████████████████████████████| 63.8MB 47kB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 1.4MB 55.9MB/s 
     |████████████████████████████████| 829kB 53.4MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 153kB 58.7MB/s 
     |████████████████████████████████| 153kB 59.9MB/s 
     |████████████████████████████████| 256kB 58.7MB/s 
     |████████████████████████████████| 276kB 57.3MB/s 
     |████████████████████████████████| 122kB 64.5MB/s 
     |████████████████████████████████| 184kB 65.0MB/s 


In [ ]:
import os
os.kill(os.getpid(), 9)

## Paso 2: Importar las librerías para el proyecto

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

## Paso 3: Análisis descriptivo del dataset


In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/joanby/tensorflow2/master/datasets/pollution-small.csv")

In [3]:
dataset.shape

(2188, 5)

In [4]:
training_data = dataset[:1600]

In [5]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [6]:
test_set = dataset[1600:]

In [7]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Paso 3b: Análisis descriptivo y validación con TFDV

### Generar análisis descriptivo del dataset


In [8]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=training_data)

### Inferir el esquema 

In [9]:
schema = tfdv.infer_schema(statistics=train_stats)

In [10]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calcular los estadísticos  descriptivos del conjunto de testing


In [11]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

## Paso 4: Comparar estadísticos del conjunto de test con Schema

### Buscar anomalías en los nuevos datos

In [12]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Mostrar todas las anomalías detectadas

- Número entero mayor que  10
- Tipo STRING cuando se esperaba un tipo INT
- Tipo FLOAT cuando se esperaba un tipo INT
- Número entero menor que 0

In [13]:
tfdv.display_anomalies(anomalies)

### Nuevos datos CON anomalías

In [14]:
test_set_copy = test_set.copy()

In [15]:
test_set_copy.drop("soot", axis=1, inplace=True)

### Estadísticos basados en datos con anomalías

In [16]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [17]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [18]:
tfdv.display_anomalies(anomalies_new)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/display_util.py:173: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Paso 5: Preparar el esquema para subir a producción

In [19]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Eliminar la columna objetivo del esquema de producción

In [20]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Comprobar anomalías entre el entorno del servidor y nuevos datos entrantes

In [21]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [22]:
tfdv.display_anomalies(serving_env_anomalies)

## Paso 6: Congelar el esquema

In [23]:
tfdv.write_schema_text(schema = schema, output_path = "pollution_schema.pbtxt")